In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from scipy import ndimage, stats
from skimage.transform import rotate

from sklearn.metrics import mean_squared_error
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import normalized_mutual_info_score

import nibabel as nib
from tqdm import tqdm
import glob
import matplotlib.pyplot as plt
import csv
import shutil
import argparse
import sys
sys.path.append("/datassd/PSMA_SEGMENTATION/Diffusion-based-Segmentation")
import random
import time
import torch
import torch as th
import torch.distributed as dist
from torchvision import transforms
from guided_diffusion import dist_util, logger
from guided_diffusion.bratsloader import PSMADataset
from guided_diffusion.script_util import (
    NUM_CLASSES,
    model_and_diffusion_defaults,
    create_model_and_diffusion,
    add_dict_to_argparser,
    args_to_dict,
)

%matplotlib inline

seed=10
th.manual_seed(seed)
th.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

os.environ["MODEL_FLAGS"] = "--image_size 256 --num_channels 128 --class_cond False --num_res_blocks 2 --num_heads 1 --learn_sigma False --use_scale_shift_norm False --attention_resolutions 16"
os.environ["DIFFUSION_FLAGS"] = "--diffusion_steps 1000 --noise_schedule linear --rescale_learned_sigmas False --rescale_timesteps False"

def read_nii_vol(target_filename):
    img = nib.load(target_filename)
    VOL = np.array(img.dataobj)
    return VOL

def visualize(img):
    _min = img.min()
    _max = img.max()
    normalized_img = (img - _min)/ (_max - _min)
    return normalized_img

def dice_score(pred, targs):
    pred = (pred>0).float()
    return 2. * (pred*targs).sum() / (pred+targs).sum()

def create_argparser():
    defaults = dict(
        data_dir="/datassd/PSMA_SEGMENTATION/DATA/testing",
        clip_denoised=True,
        num_samples=1,
        batch_size=1,
        use_ddim=False, #False
        model_path="",
        num_ensemble=10      #number of samples in the ensemble
    )
    defaults.update(model_and_diffusion_defaults())
    parser = argparse.ArgumentParser("")
    add_dict_to_argparser(parser, defaults)
    return parser

def normalize(img, _min, _max):
    normalized_img = (img - _min)/ (_max - _min)
    return normalized_img

def normalize3(volume):
    min_val = volume.min()
    max_val = volume.max()
    normalized_volume = (volume - min_val) / (max_val - min_val)
    return normalized_volume


args = create_argparser().parse_args("")
dist_util.setup_dist('3')
logger.configure()

args.image_size = 256 
args.num_channels = 128 
args.class_cond = False 
args.num_res_blocks = 2 
args.num_heads = 1 
args.learn_sigma = False 
args.use_scale_shift_norm = False 
args.attention_resolutions = "16"
args.diffusion_steps = 1000
args.noise_schedule = "linear" 
args.rescale_learned_sigmas = False 
args.rescale_timesteps = False
args.data_dir = "/datassd/PSMA_SEGMENTATION/DATA/testing"  
args.model_path = "/datassd/PSMA_SEGMENTATION/Diffusion-based-Segmentation/results/savedmodel1500000.pt" 
args.num_ensemble = 10


logger.log("creating model and diffusion...")
model, diffusion = create_model_and_diffusion(
    **args_to_dict(args, model_and_diffusion_defaults().keys())
)

print(f"args.model_path: {args.model_path}")

model.load_state_dict(
    dist_util.load_state_dict(args.model_path, map_location="cpu")
)

model.to(dist_util.dev())

if args.use_fp16:
    model.convert_to_fp16()
    
model.eval()

logger.log("sampling...")

start = th.cuda.Event(enable_timing=True)
end = th.cuda.Event(enable_timing=True)

model_kwargs = {}
start.record()

sample_fn = (
    diffusion.p_sample_loop_known if not args.use_ddim else diffusion.ddim_sample_loop_known
)



/data/anaconda/envs/diff/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Logging to ./results
creating model and diffusion...
args.model_path: /datassd/PSMA_SEGMENTATION/Diffusion-based-Segmentation/results/savedmodel1500000.pt
sampling...


In [2]:
dataset_root = "/datassd/PSMA_SEGMENTATION/DATA"
test_dir = os.path.join(dataset_root, "testing")
save_dir = os.path.join(dataset_root, "inferences")

index = 0
test_patients_dir_list = os.listdir(test_dir)
for study in test_patients_dir_list[1800*3:1800*4]:
    study_name = os.path.join(test_dir, study)

    folder_name = study_name.split("/")[-1]
    top_bottom = folder_name[-2:]
    rotation = folder_name[-6:-3]
    top_bottom_str = "Top" if top_bottom == '01' else "Bottom"
    print(f"Patient ID: {folder_name[:-7]} | Body part: {top_bottom_str} | Rotation: {rotation}")
    
    # '/datassd/PSMA_SEGMENTATION/DATA/testing/PSMA-01-097-000-02/PSMA-01-097_PETD_000-02.nii.gz'
    gt_filename = f"/datassd/PSMA_SEGMENTATION/DATA/testing/{folder_name[:-7]}-{rotation}-{top_bottom}/{folder_name[:-7]}_SEG_{rotation}-{top_bottom}.nii.gz"
    gt = read_nii_vol(gt_filename)
    gt = np.uint8(gt)
    
    if gt.max() == 0:
        target_dir = os.path.join(save_dir, folder_name)
        if not os.path.exists(target_dir):
            os.mkdir(target_dir)
        seg_filename = f"{folder_name[:-7]}_Predicted-Seg_{rotation}-{top_bottom}.npy"
        empty_mask = np.zeros((10,1,250,250), dtype=np.float32)
        np.save(os.path.join(target_dir, seg_filename), empty_mask)
        
    else:
        mip_filename = f"/datassd/PSMA_SEGMENTATION/DATA/testing/{folder_name[:-7]}-{rotation}-{top_bottom}/{folder_name[:-7]}_PET_{rotation}-{top_bottom}.nii.gz"
        mip = read_nii_vol(mip_filename)
        mip = np.float32(mip)
        
        torch_gt = th.tensor(gt)

        seqtypes = ['PET', 'PET10', 'PET5', 'PETD']
        out = []
        for seqtype in seqtypes:
            _img = read_nii_vol( f"/datassd/PSMA_SEGMENTATION/DATA/testing/{folder_name[:-7]}-{rotation}-{top_bottom}/{folder_name[:-7]}_{seqtype}_{rotation}-{top_bottom}.nii.gz")
            out.append(torch.tensor(_img.astype(np.float32)))

        out = torch.stack(out)
        out = out.unsqueeze(0)
        out = torch.nn.functional.pad(out, pad=(3,3,3,3), mode="constant")

        c = th.randn_like(out[:, :1, ...])
        img = th.cat((out, c), dim=1)     #add a noise channel$ 

        fig_size = (20, 9)
        images_cmap = "bone" #"Greys_r"

        samples_list = list()

        # fig, axes = plt.subplots(1, 2, figsize=fig_size)
        # axes[0].pcolormesh(mip.T, cmap=images_cmap)
        # axes[0].axis('off')
        # axes[0].set_title('MIP Image')

        # axes[1].pcolormesh(gt.T, cmap=images_cmap)
        # axes[1].axis('off')
        # axes[1].set_title('GT Mask')
        # plt.show()

        for i in tqdm(range(args.num_ensemble)):
            model_kwargs = {}
            start.record()
            sample_fn = (
                diffusion.p_sample_loop_known if not args.use_ddim else diffusion.ddim_sample_loop_known
            )
            
            sample, x_noisy, org = sample_fn(
                model,
                (args.batch_size, 4, args.image_size, args.image_size), img,
                clip_denoised=args.clip_denoised,
                model_kwargs=model_kwargs,
            )

            end.record()
            th.cuda.synchronize()
            # print('time for 1 sample', start.elapsed_time(end))  #time measurement for the generation of 1 sample

            _sample = sample[0,...]
            __sample = _sample[:, 3:-3, 3:-3]
            samples_list.append(__sample.cpu().numpy())
        
        array_stack = np.stack(samples_list)
        # mean_image = np.mean(array_stack, axis=0)
        # std_image = np.std(array_stack, axis=0)
        # var_image = np.var(array_stack, axis=0)

        # plt.figure(figsize=(26, 30))
        # images_cmap = 'bone'
        # plt.subplot(4, 3, 1)
        # plt.pcolormesh(samples_list[0][0,:,:].T, cmap=images_cmap, vmin=0, vmax=1)
        # plt.axis('off')
        # plt.title('Sample 1')
        # plt.colorbar()

        # plt.subplot(4, 3, 2)
        # plt.pcolormesh(samples_list[1][0,:,:].T, cmap=images_cmap, vmin=0, vmax=1)
        # plt.axis('off')
        # plt.title('Sample 2')
        # plt.colorbar()

        # plt.subplot(4, 3, 3)
        # plt.pcolormesh(samples_list[2][0,:,:].T, cmap=images_cmap, vmin=0, vmax=1)
        # plt.axis('off')
        # plt.title('Sample 3')
        # plt.colorbar()

        # plt.subplot(4, 3, 4)
        # plt.pcolormesh(samples_list[3][0,:,:].T, cmap=images_cmap, vmin=0, vmax=1)
        # plt.axis('off')
        # plt.title('Sample 4')
        # plt.colorbar()

        # plt.subplot(4, 3, 5)
        # plt.pcolormesh(samples_list[4][0,:,:].T, cmap=images_cmap, vmin=0, vmax=1)
        # plt.axis('off')
        # plt.title('Sample 5')
        # plt.colorbar()

        # plt.subplot(4, 3, 6)
        # plt.pcolormesh(samples_list[5][0,:,:].T, cmap=images_cmap, vmin=0, vmax=1)
        # plt.axis('off')
        # plt.title('Sample 6')
        # plt.colorbar()

        # plt.subplot(4, 3, 7)
        # plt.pcolormesh(samples_list[6][0,:,:].T, cmap=images_cmap, vmin=0, vmax=1)
        # plt.axis('off')
        # plt.title('Sample 7')
        # plt.colorbar()

        # plt.subplot(4, 3, 8)
        # plt.pcolormesh(samples_list[7][0,:,:].T, cmap=images_cmap, vmin=0, vmax=1)
        # plt.axis('off')
        # plt.title('Sample 8')
        # plt.colorbar()

        # plt.subplot(4, 3, 9)
        # plt.pcolormesh(samples_list[8][0,:,:].T, cmap=images_cmap, vmin=0, vmax=1)
        # plt.axis('off')
        # plt.title('Sample 9')
        # plt.colorbar()

        # plt.subplot(4, 3, 10)
        # plt.pcolormesh(samples_list[9][0,:,:].T, cmap=images_cmap, vmin=0, vmax=1)
        # plt.axis('off')
        # plt.title('Sample 10')
        # plt.colorbar()

        # plt.subplot(4, 3, 11)
        # plt.pcolormesh(mean_image[0,:,:].T, cmap='viridis', vmin=0, vmax=1)
        # plt.axis('off')
        # plt.title('mean')
        # plt.colorbar()

        # plt.subplot(4, 3, 12)
        # plt.pcolormesh(var_image[0,:,:].T, cmap='viridis', vmin=0, vmax=1)
        # plt.axis('off')
        # plt.title('std')
        # plt.colorbar()
        # plt.show()





        # plt.figure(figsize=(20, 5))
        # plt.subplot(1, 3, 1)
        # plt.pcolormesh(mip.T, cmap='bone')
        # plt.axis('off')
        # plt.title('MIP')
        # plt.colorbar()

        # plt.subplot(1, 3, 2)
        # plt.pcolormesh(gt.T, cmap='bone')
        # plt.axis('off')
        # plt.title('GT')
        # plt.colorbar()

        # plt.subplot(1, 3, 3)
        # plt.pcolormesh(gt.T - mean_image[0,:,:].T, cmap='bwr', vmin=-1, vmax=1)
        # plt.axis('off')
        # plt.title('GT - Mean')
        # plt.colorbar()
        # plt.show()
        
        target_dir = os.path.join(save_dir, folder_name)
        if not os.path.exists(target_dir):
            os.mkdir(target_dir)
        
        seg_filename = f"{folder_name[:-7]}_Predicted-Seg_{rotation}-{top_bottom}.npy"
        np.save(os.path.join(target_dir, seg_filename), array_stack)
        
        asghar = "agha"
    index += 1

Patient ID: PSMA-01-534 | Body part: Top | Rotation: 180


100%|██████████| 10/10 [09:06<00:00, 54.64s/it]


Patient ID: PSMA-01-534 | Body part: Bottom | Rotation: 180


100%|██████████| 10/10 [09:06<00:00, 54.67s/it]


Patient ID: PSMA-01-534 | Body part: Top | Rotation: 185


...

100%|██████████| 10/10 [08:56<00:00, 53.64s/it]


Patient ID: PSMA-01-281 | Body part: Top | Rotation: 350
Patient ID: PSMA-01-281 | Body part: Bottom | Rotation: 350


100%|██████████| 10/10 [08:44<00:00, 52.43s/it]


Patient ID: PSMA-01-281 | Body part: Top | Rotation: 355
Patient ID: PSMA-01-281 | Body part: Bottom | Rotation: 355


100%|██████████| 10/10 [08:50<00:00, 53.03s/it]
